In [ ]:
# default_exp im2im_annotator

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nbdev import *

In [ ]:
#exporti

from functools import partial

from ipywidgets import (AppLayout, VBox, HBox, Button, GridBox, Layout, Checkbox, HTML, IntText, Valid, Output, Image)
from ipyevents import Event
from ipycanvas import Canvas, hold_canvas

from pathlib import Path

from ipyannotator.navi_widget import Navi
from ipyannotator.storage import setup_project_paths, get_image_list_from_folder, AnnotationStorage

from traitlets import Dict, Int, Float, HasTraits, observe, dlink, link, List, Unicode

from ipyannotator.capture_annotator import CaptureGrid
from ipyannotator.image_button import ImageButton
from ipyannotator.bbox_canvas import draw_img


In [ ]:
# exporti

class ImCanvas(HBox, HasTraits):
    image_path = Unicode()
    _image_scale = Float()
    
    def __init__(self, width=150, height=150):
        
        self._canvas = Canvas(width=width, height=height)

        super().__init__([self._canvas])  

    @observe('image_path')
    def _draw_image(self, change):
        self._image_scale = draw_img(self._canvas, self.image_path, clear=True)

    # Add value as a read-only property
    @property
    def image_scale(self):
        return self._image_scale
    
    def _clear_image(self):
        self._canvas.clear()
        
    # needed to support voila
    # https://ipycanvas.readthedocs.io/en/latest/advanced.html#ipycanvas-in-voila
    def observe_client_ready(self, cb=None):
        self._canvas.on_client_ready(cb)

In [ ]:
im = ImCanvas(35, 35)
im.image_path = '../data/projects/im2im1/class_images/blocks_1.png'
display(im)
im.image_scale

In [ ]:
im._clear_image()
im.image_scale

In [ ]:
#exporti

class Im2ImAnnotatorGUI(AppLayout):
    def __init__(self, im_width=300, im_height=300, 
                       label_width=150, label_height=150, 
                       n_rows=3, n_cols=3, label_autosize=False):
        
        
        if label_autosize:
            n_rows = 4 
            n_cols = 4
            if im_width <100 or im_height < 100:
                label_width = 10
                label_height = 10
            elif im_width >1000 or im_height > 1000:
                label_width = 50
                label_height = 10
            else:
                label_width = min(im_width, im_height)/10
                label_height = label_width
                
        self.label_width = label_width
        self.label_height = label_height
        self.n_rows = n_rows
        self.n_cols = n_cols
        
        self._image = ImCanvas(width=im_width, height=im_height)
            
        self._navi = Navi()
        
        self._save_btn = Button(description="Save",
                                layout=Layout(width='auto'))
        
        
        self._controls_box = HBox([self._navi, self._save_btn],
                                 layout=Layout(display='flex', justify_content='center', flex_flow='wrap', align_items='center'))
        
         
        self._grid_box = CaptureGrid(grid_item=ImageButton, image_width=label_width, image_height=label_height,  n_rows=n_rows, n_cols=n_cols)


        self._grid_label = HTML()
        self._labels_box = VBox(children = [self._grid_label, self._grid_box],
                                layout=Layout(display='flex', justify_content='center', flex_wrap='wrap', align_items='center'))

        
        super().__init__(header=None,
                 left_sidebar=VBox([self._image,self._controls_box], layout=Layout(display='flex', justify_content='center', flex_wrap='wrap', align_items='center')),
                 center=self._labels_box,
                 right_sidebar=None,
                 footer=None,
                 pane_widths=(6, 4, 0),
                 pane_heights=(1, 1, 1))
        
    def on_client_ready(self, callback):
        self._image.observe_client_ready(callback)
        

In [ ]:
label_state = {
    '../data/projects/im2im1/class_images/blocks_1.png': {'answer': False}, 
    '../data/projects/im2im1/class_images/blocks_9.png': {'answer': False},
    '../data/projects/im2im1/class_images/blocks_12.png': {'answer': True},
    '../data/projects/im2im1/class_images/blocks_32.png': {'answer': False},
    '../data/projects/im2im1/class_images/blocks_37.png': {'answer': False},
    '../data/projects/im2im1/class_images/blocks_69.png': {'answer': True}
}

In [ ]:
im2im_ = Im2ImAnnotatorGUI(im_height = 500, im_width = 500, label_width=50, label_height=50, n_rows=2, n_cols=3)
im2im_._image.image_path='../data/projects/im2im1/pics/Grass1.png'
im2im_

In [ ]:
im2im_._grid_box.current_state = label_state

In [ ]:
#exporti
def _label_state_to_storage_format(label_state):
    return [Path(k).name for k, v in label_state.items() if v['answer']]

In [ ]:
label_state_storage = _label_state_to_storage_format(label_state); label_state_storage

In [ ]:
#exporti
def _storage_format_to_label_state(storage_format, label_names, label_dir):
    return {str(Path(label_dir)/label): {'answer': label in storage_format} for label in label_names}

In [ ]:
label_names = ['blocks_1.png', 'blocks_9.png', 'blocks_12.png', 'blocks_32.png', 'blocks_37.png', 'blocks_69.png']
restored_label_state = _storage_format_to_label_state(label_state_storage, label_names, '../data/projects/im2im1/class_images/')
test_eq(label_state, restored_label_state)

In [ ]:
#exporti

class Im2ImAnnotatorLogic(HasTraits):
    debug_output = Output(layout={'border': '1px solid black'})
    index = Int(0) # state index
    image_path = Unicode() # current image path
    current_im_num = Int()
    disp_number = Int() # number of labels on screen
    label_state = Dict()
    question_value = Unicode()

    
    def __init__(self, project_path, question=None, image_dir='pics', label_dir='class_images'):
        
        self.project_path = Path(project_path)
        self.image_dir, self.annotation_file_path = setup_project_paths(self.project_path, image_dir=image_dir)
        self.image_paths = sorted(get_image_list_from_folder(self.image_dir))
        self.current_im_num = len(self.image_paths)
        
        self.labels_im_dir = Path(self.project_path, 'class_images')
        self.labels_files = sorted(get_image_list_from_folder(self.labels_im_dir, strip_path=True))
        
        self.annotations = AnnotationStorage(self.image_paths)
        
        if question:
            self.question_value = f'<center><p style="font-size:20px;">{question}</p></center>'
        
        
    def _update_im(self):
        self.image_path = str(self.image_paths[self.index])
        
        
    def _update_state(self, change=None): # from annotations
        im_name = Path(self.image_path).name
        
        current_annotation = self.annotations[im_name] if im_name else None
            
        self.label_state = _storage_format_to_label_state(storage_format=current_annotation or [],
                                                     label_names=self.labels_files,
                                                     label_dir=self.labels_im_dir)


    def _update_annotations(self, index): # from screen
        self.annotations[Path(self.image_path).name] = _label_state_to_storage_format(self.label_state)

        
    def _save_annotations(self, *args, **kwargs): # to disk
        index = kwargs.pop('old_index', self.index)
        self._update_annotations(index)          
        self.annotations.save(self.annotation_file_path)
        
        
    @observe('index')
    def _idx_changed(self, change):
        ''' On index change save old state 
            and update current state for visualisation
        '''
        self._save_annotations(old_index = change['old'])
        # update new screen
        self._update_im()
        self._update_state()
        

        
    @debug_output.capture(clear_output=False)
    def _handle_grid_click(self, event, name=None):
        label_changed = Path(self.labels_im_dir,  name)
        
        if label_changed.is_dir():
            # button without image - invalid
            return
        
        label_changed = str(label_changed)
        current_label_state = self.label_state.copy()
        
        # inverse state
        current_label_state[label_changed] = {'answer': not self.label_state[label_changed].get('answer', False)}
        
        # change traitlets.Dict entirely to have change events issued
        self.label_state = current_label_state
        

    def _handle_client_ready(self):
        self._update_im()
        self._update_state()

In [ ]:
i_ = Im2ImAnnotatorLogic(project_path='../data/projects/im2im1/')
# cal.disp_number = 9 # should be synced from gui 
i_.image_dir, i_.annotation_file_path, i_.labels_im_dir, i_.current_im_num, 

##### (Next-> button emulation) 
Increment index to initiate annotation save and switch state for a new screen

In [ ]:
i_.index=2
display(i_.label_state)

In [ ]:
#export

class Im2ImAnnotator(Im2ImAnnotatorGUI):
    """
    Represents image to image annotator. 
    
    Gives an ability to itarate through image dataset, 
    matching images with image labels,
    export final annotations in json format
    
    """
    
    def __init__(self, project_path, image_dir=None, im_width=100, im_height=100, 
                 label_width=150, label_height=150, 
                 n_rows=3, n_cols=3, label_autosize=False, question=None):
        
        super().__init__(im_width, im_height, label_width, label_height, n_rows, n_cols, label_autosize)
        
        self._model = Im2ImAnnotatorLogic(project_path, question, image_dir)
        
        self._save_btn.on_click(self._model._save_annotations)        

        self._grid_box.on_click(self._model._handle_grid_click)
            
        # set correct slider max value based on image number
        dlink((self._model, 'current_im_num'), (self._navi.model, 'max_im_number'))
        
        # link current image index from controls to annotator model 
        link((self._navi.model, 'index'), (self._model, 'index'))
        
        # link annotation question 
        link((self._model, 'question_value'), (self._grid_label, 'value'))
        
        #link image vizualizer
        dlink((self._model, 'image_path'), (self._image, 'image_path'))
        
        # draw current image and bbox only when client is ready
        self.on_client_ready(self._model._handle_client_ready)
        
        # link state of model and grid box visualizer
        link((self._model, 'label_state'), (self._grid_box, 'current_state'))
        
            
    def to_dict(self, only_annotated=True):
        return self._model.annotations.to_dict(only_annotated)

In [ ]:
# im2im = Im2ImAnnotator('../data/projects/im2im1', 'pics', 200,200, 50, 50, 3, 4, question="HelloWorld")
im2im = Im2ImAnnotator('../data/projects/im2im1', 'pics', 400, 400, label_autosize=True, question="HelloWorld")

im2im

In [ ]:
im2im.to_dict()

In [ ]:
im2im._model.debug_output

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()